<a href="https://colab.research.google.com/github/alfredofosu/python.projects/blob/main/_04HyperparameterOptimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Setup a virtual environemnt
from IPython.display import clear_output
# !pip install --upgrade pip
# !pip install virtualenv
# !virtualenv /content/drive/MyDrive/Colab_Notebooks/colab_env

# import sys
# # activate virtual environment
# !source /content/drive/MyDrive/Colab_Notebooks/colab_env/bin/activate
# #  add virtual environment path to colab's system path
# sys.path.append("/content/drive/MyDrive/Colab_Notebooks/colab_env/lib/python3.10/site-packages")
clear_output(wait=False)

# Setting up Cloud Storage

In [2]:
from google.colab import auth
PROJECT_ID = "master-thesis-yorku-aofosu"  # @param {type:"string"}

auth.authenticate_user(project_id=PROJECT_ID)

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

!mkdir colab_bucket
!gcsfuse --implicit-dirs master-thesis-yorku-aofosu colab_bucket

clear_output(wait=False)
!ls /content/colab_bucket

hyperOpt


# Libraries and Connections to Bucket

In [4]:
#@title Install Libraries
%pip install optuna
%pip install optuna.integration
%pip install pyyaml h5py
%pip install tensorflow==2.13.*

%pip install lets-plot
%pip install cairosvg

%pip install scikeras[tensorflow]
%pip install -q "git+https://github.com/tensorflow/docs"
%pip install -q -U keras-tuner
clear_output(wait=False)

In [5]:
#@title Import Libraries

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

# Data transformation libraries
import pandas as pd

# Numeric manipulation libraries
import numpy as np
np.set_printoptions(precision=3, suppress=True)
import random

# Other libraries
from tqdm import tqdm

# Date and time manipulation libraries
from datetime import datetime
import time
import pytz
eastern = pytz.timezone('Canada/Eastern')

# Hyperparameter tuning libraries
import keras_tuner as kt
# import optuna

# Deep learning libraries
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots

#----------------------------------------------------------------------------------
# Instantiating a GPU or TPU
# try:
#   tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
#   print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
#   tf.config.experimental_connect_to_cluster(tpu)
#   tf.tpu.experimental.initialize_tpu_system(tpu)
#   strategy = tf.distribute.TPUStrategy(tpu)
#   print("Number of TPU devices: ",  len(tf.config.list_logical_devices("TPU")))
#   print("Number of accelerators: ", strategy.num_replicas_in_sync)
# except ValueError:  # detect GPUs
#   strategy = tf.distribute.MirroredStrategy() # for GPU or multi-GPU machines
#   if tf.test.gpu_device_name() != '/device:GPU:0':
#     print('WARNING: GPU device not found.')
#   else:
#     print('SUCCESS: Found GPU: {}'.format(tf.test.gpu_device_name()))
#   print("Number of accelerators: ", strategy.num_replicas_in_sync)
#   # raise BaseException('ERROR: Not connected to a TPU runtime!'
#----------------------------------------------------------------------------------

# Python in-built libraries
from platform import python_version
print("Python version:", python_version())

# Google colab libraries
from google.colab import runtime

# Data visualization libraries
from lets_plot import *
LetsPlot.setup_html()

from pathlib import Path

# Set a fixed random seed to always get the same results
SEED = 42
random.seed(SEED)
tf.random.set_seed(SEED)
np.random.seed(SEED)

TensorFlow version: 2.13.0
Python version: 3.10.12


In [6]:
# set local workspace
local_path = "/content/drive/MyDrive/Colab_Notebooks/master_thesis" # @param {type:"string"}


In [7]:
# #@title Copy files to the bucket
# !gsutil -m cp -r "/content/drive/MyDrive/Colab Notebooks/master_thesis/hyperOpt/all/tensorboard/tb_logs/mse/sLnn/logs" gs://master-thesis-yorku-aofosu/hyperOpt/all/mse/sLnn

# Load and preprocess data

In [8]:
#@title Load dataset

# A function to load a timeseries dataset
def load(path_to_dataset: str, date_column_name: str) -> pd.DataFrame:

  '''
  @param path_to_dataset: a path where a timeseries dataset is stored -> Pandas Dataframe
  @param date_column_name: a column containing date values -> String

  '''
  df = pd.read_csv(path_to_dataset, parse_dates=[date_column_name])

  df[date_column_name] = pd.to_datetime(df[date_column_name]) # <- utc=True, if needed

  df.set_index(date_column_name, inplace=True)

  # Assuming 'df' is your DataFrame
  numeric_columns = df.select_dtypes(include=['int', 'float']).columns

  # Convert numeric columns to float32
  df[numeric_columns] = df[numeric_columns].astype("float32")

  # Convert columns with data type 'object' to 'category'
  object_columns = df.select_dtypes(['object']).columns
  df[object_columns] = df[object_columns].astype('category')

  df = df.rename(columns={'Ca2+':'Ca', 'Cl-':'Cl', 'K+':'K', 'Mg2+':'Mg', 'Na+':'Na', 'SO42-':'SO42',
                          'CO32-':'CO32', 'HCO3-': 'HCO3'})

  return df

# Load 1st Approach Data - Predicting each major ion with the combined dataset
dataset = load(f'{local_path}/data/modelling_data/wq_all_unscaled.csv', 'date')

# Load Second Dataset - Up predicts Down
dataset_x = load(f'{local_path}/data/modelling_data/wq_up_down_unscaled.csv', 'date')
dataset_x.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1368 entries, 1965-01-31 to 2021-12-31
Data columns (total 55 columns):
 #   Column                             Non-Null Count  Dtype   
---  ------                             --------------  -----   
 0   major_basin_X                      1368 non-null   category
 1   Ca_X                               1368 non-null   float32 
 2   Cl_X                               1368 non-null   float32 
 3   K_X                                1368 non-null   float32 
 4   Mg_X                               1368 non-null   float32 
 5   Na_X                               1368 non-null   float32 
 6   SO42_X                             1368 non-null   float32 
 7   TDS_X                              1368 non-null   float32 
 8   ALKT_X                             1368 non-null   float32 
 9   CNDT_X                             1368 non-null   float32 
 10  HARD_X                             1368 non-null   float32 
 11  pH_X                     

In [10]:
dataset_x.tail(13)

,major_basin_X,Ca_X,Cl_X,K_X,Mg_X,Na_X,SO42_X,TDS_X,ALKT_X,CNDT_X,...,avg_water_Temp_f_X,CCMEWQI_X,rank_X,Ca_y,Cl_y,K_y,Mg_y,Na_y,SO42_y,TDS_y
date,,,,,,,,,,,,,,,,,,,,,
2020-12-31,East Don River,111.500000,1430.000000,3.265,17.200001,849.50,57.150002,2857.00,209.000000,4805.25,...,0.0,64.941902,Marginal,85.800003,572.000000,3.870000,15.600000,392.000000,61.532845,1555.000000
2021-01-31,East Don River,139.500000,988.000000,3.140,25.450001,577.50,69.650002,2053.00,291.445007,3358.50,...,0.0,70.604431,Fair,131.000000,1290.000000,5.310000,25.600000,844.000000,44.223610,2642.000000
2021-02-28,East Don River,164.000000,2405.000000,6.665,24.049999,1385.00,117.949997,4393.25,182.054993,7231.00,...,0.0,57.866955,Marginal,103.000000,1760.000000,4.120000,14.800000,1200.000000,68.305794,4179.000000
2021-03-31,East Don River,119.000000,501.000000,3.035,23.549999,264.50,61.549999,1299.75,260.475006,2116.25,...,0.0,80.404762,Good,100.000000,577.000000,4.450000,23.799999,321.000000,53.643860,1497.000000
2021-04-30,East Don River,98.500000,860.000000,3.095,17.000000,495.00,57.750000,1931.00,194.330002,3172.00,...,0.0,72.761642,Fair,95.800003,564.000000,3.600000,19.100000,323.000000,76.079018,1435.000000
2021-05-31,East Don River,127.000000,358.000000,0.825,25.000000,182.50,53.000000,1018.00,280.589996,1671.50,...,1.0,83.317642,Good,97.400002,377.000000,5.000000,19.500000,206.000000,42.442326,1093.000000
2021-06-30,East Don River,115.500000,298.000000,2.785,21.400000,157.50,46.450001,923.00,256.600006,1469.25,...,1.0,84.341774,Good,87.599998,340.000000,4.670000,18.100000,183.000000,30.695601,953.000000
2021-07-31,East Don River,106.000000,256.000000,2.890,20.600000,130.00,43.950001,870.25,242.899994,1394.75,...,1.0,84.783707,Good,103.190697,386.828918,4.944576,18.557232,149.029083,81.511070,1397.144043
2021-08-31,East Don River,111.500000,222.500000,2.965,23.100000,116.00,46.250000,751.00,249.100006,1259.00,...,1.0,85.564262,Good,74.166664,216.333328,3.673333,16.250000,144.566666,25.400000,727.333313


In [11]:
# Group the parameters and features
parameters = {'water quality parameters': ['Ca', 'Cl', 'K', 'Mg', 'Na','SO42', 'TDS',],
              'water quality parameters x': ['Ca_y', 'Cl_y', 'K_y', 'Mg_y', 'Na_y','SO42_y', 'TDS_y',],
              }

In [132]:
# dataset_to_model = dataset[['major_basin',
#                             'Ca', 'Cl', 'K', 'Mg', 'Na','SO42', 'TDS',
#                             'CO32', 'HCO3', 'ALKT', 'CNDT', 'HARD', 'pH','avg_water_Temp',
#                             'avg_air_temp', 'avg_precip', 'avg_discharge', 'avg_stage', 'max_Wx', 'max_Wy',
#                             'Ca_f', 'Cl_f', 'Na_f', 'SO42_f', 'pH_max_f', 'pH_min_f', 'avg_water_Temp_f',
#                             'CCMEWQI', 'rank',
#                             'missingindicator_Ca2+', 'missingindicator_Cl-', 'missingindicator_K+',
#                             'missingindicator_Mg2+', 'missingindicator_Na+',
#                             'missingindicator_SO42-', 'missingindicator_TDS',
#                             'missingindicator_ALKT', 'missingindicator_CNDT',
#                             'missingindicator_HARD', 'missingindicator_pH',
#                             'missingindicator_avg_water_Temp', 'missingindicator_avg_discharge',
#                             'missingindicator_avg_stage', 'missingindicator_avg_air_temp',
#                             'missingindicator_avg_precip', 'missingindicator_max_Wx',
#                             'missingindicator_max_Wy',
#                             ]].copy()

dataset_to_model_x = dataset_x[['major_basin_X', 'Ca_X', 'Cl_X', 'K_X', 'Mg_X', 'Na_X','SO42_X', 'TDS_X',
                                'ALKT_X', 'CNDT_X', 'HARD_X', 'pH_X', 'CO32_X', 'HCO3_X',
                                'avg_water_Temp_X', 'avg_discharge_X', 'avg_stage_X',
                                'avg_air_temp_X', 'avg_precip_X', 'max_Wx_X', 'max_Wy_X',
                                # 'missingindicator_Ca_X', 'missingindicator_Cl_X',
                                # 'missingindicator_K_X', 'missingindicator_Mg_X',
                                # 'missingindicator_Na_X', 'missingindicator_SO42_X',
                                # 'missingindicator_TDS_X', 'missingindicator_ALKT_X',
                                # 'missingindicator_CNDT_X', 'missingindicator_HARD_X',
                                # 'missingindicator_pH_X',
                                # 'missingindicator_avg_water_Temp_X',
                                # 'missingindicator_avg_discharge_X', 'missingindicator_avg_stage_X',
                                # 'missingindicator_avg_air_temp_X', 'missingindicator_avg_precip_X',
                                # 'missingindicator_max_Wx_X', 'missingindicator_max_Wy_X',
                                # 'Ca_f_X','Cl_f_X', 'Na_f_X', 'SO42_f_X', 'pH_max_f_X', 'pH_min_f_X','avg_water_Temp_f_X',
                                # 'CCMEWQI_X',
                                'Ca_y', 'Cl_y', 'K_y','Mg_y', 'Na_y', 'SO42_y', 'TDS_y',]].copy()

# dataset_to_model = dataset_to_model.groupby('date').mean()
# dataset_to_model.info()
# dataset_to_model_x.info()

In [13]:
dataset_to_model_x.describe()

,Ca_X,Cl_X,K_X,Mg_X,Na_X,SO42_X,TDS_X,ALKT_X,CNDT_X,HARD_X,...,pH_min_f_X,avg_water_Temp_f_X,CCMEWQI_X,Ca_y,Cl_y,K_y,Mg_y,Na_y,SO42_y,TDS_y
count,1368.000000,1368.000000,1368.000000,1368.000000,1368.000000,1368.000000,1368.000000,1368.000000,1368.000000,1368.000000,...,1368.0,1368.000000,1368.000000,1368.000000,1368.000000,1368.000000,1368.000000,1368.000000,1368.000000,1368.000000
mean,105.080170,362.850616,3.535991,17.596149,212.923615,49.203148,1011.477051,212.142319,1637.363281,337.739685,...,0.0,0.277047,85.731232,97.660873,344.450256,3.751671,16.737942,200.703644,49.947311,898.626831
std,23.282228,363.382294,1.590683,4.299785,195.785294,14.172410,652.732788,44.709831,1128.829102,76.192490,...,0.0,0.447703,9.137426,24.917759,394.359039,1.554919,4.585690,222.854507,17.064468,598.139343
min,21.092251,0.000000,0.170104,2.392559,0.000000,8.318313,11.247286,62.843636,42.457413,39.155308,...,0.0,0.000000,53.753723,0.113000,0.000000,0.003100,0.020100,0.000000,5.914264,25.441805
25%,89.454746,118.375000,2.550000,14.532604,91.889584,39.903006,586.162552,185.379574,909.773849,287.069656,...,0.0,0.000000,78.985514,80.251501,133.808731,2.749619,13.756427,72.178301,37.803691,542.812500
50%,104.993031,233.392815,3.366624,17.536710,164.935104,48.652571,850.095093,213.010849,1285.447998,337.880554,...,0.0,0.000000,85.908348,96.869648,204.000000,3.818478,16.744036,141.783073,49.502325,709.535919
75%,119.365871,534.327866,4.407233,20.364126,288.819977,57.714706,1315.555298,243.944450,2126.954407,387.001335,...,0.0,1.000000,93.025692,114.000000,392.749786,4.693509,19.832062,244.960835,60.872105,1082.120544
max,194.763153,3380.000000,20.000000,32.303604,2090.000000,117.949997,5449.500000,448.481049,11965.000000,621.632751,...,0.0,1.000000,100.000000,176.800690,3920.000000,12.500000,32.381100,1972.887695,178.303329,4840.000000


In [133]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn import preprocessing
from sklearn.pipeline import Pipeline, make_pipeline

#Normalizer
def normalizer(x: int, ):

  normalizer_dict = {
      1: preprocessing.StandardScaler(),
      2: preprocessing.MinMaxScaler(),
      3: preprocessing.RobustScaler(),
      4: preprocessing.PowerTransformer(),
      5: preprocessing.QuantileTransformer(output_distribution='normal', ),
  }
  return normalizer_dict.get(x)


norm_pipeline = Pipeline(steps=[
      ("transformer", normalizer(5)),
      ("scaler", normalizer(2)),
])

# columns_to_scale = ['Ca', 'Cl', 'K', 'Mg', 'Na','SO42', 'TDS',
#                     'CO32', 'HCO3', 'ALKT', 'CNDT', 'HARD', 'pH','avg_water_Temp',
#                     'avg_air_temp', 'avg_precip', 'avg_discharge', 'avg_stage', 'max_Wx', 'max_Wy',
#                     'CCMEWQI']

columns_to_scale_x = ['Ca_X', 'Cl_X', 'K_X', 'Mg_X', 'Na_X','SO42_X', 'TDS_X',
                      'ALKT_X', 'CNDT_X', 'HARD_X', 'pH_X', 'CO32_X', 'HCO3_X',
                      'avg_water_Temp_X', 'avg_discharge_X', 'avg_stage_X',
                      'avg_air_temp_X', 'avg_precip_X',
                      # 'CCMEWQI_X',
                      'Ca_y', 'Cl_y', 'K_y', 'Mg_y', 'Na_y', 'SO42_y', 'TDS_y']
categorical_columns = [
    # "major_basin",
    # "rank",
    # "date_year",
    # "date_season",
  ]
categories=[
    # ['West Don River', 'East Don River', 'Lower Don River'],
    # ["Poor", "Marginal", "Fair", "Good", "Excellent",],
    # ['1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1974',
    #  '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984',
    #  '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994',
    #  '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
    #  '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
    #  '2015', '2016', '2017', '2018', '2019', '2020', '2021'],
    # ["winter","spring", "summer", "autumn",],
    ]
# ordinal_encoder = OrdinalEncoder(handle_unknown='error', categories=categories)
one_hot_encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)

pipeline = make_pipeline (
    ColumnTransformer(
        transformers=[
            # ("categorical", ordinal_encoder, categorical_columns),
            ("one-hot-encoder", one_hot_encoder, ['major_basin_X']),
            ("numerical", normalizer(1), columns_to_scale_x)
        ],
        remainder="passthrough",
        verbose_feature_names_out=False,
    )
).set_output(transform="pandas")

# dataset_scaled = pipeline.fit_transform(dataset_to_model)
dataset_scaled_x = pipeline.fit_transform(dataset_to_model_x)

In [129]:
dataset_scaled_x

,major_basin_X_East Don River,major_basin_X_West Don River,Ca_X,Cl_X,K_X,Mg_X,Na_X,SO42_X,TDS_X,ALKT_X,...,avg_precip_X,Ca_y,Cl_y,K_y,Mg_y,Na_y,SO42_y,TDS_y,max_Wx_X,max_Wy_X
date,,,,,,,,,,,,,,,,,,,,,
1965-01-31,0.0,1.0,-0.086246,-0.556061,-0.098163,-0.560814,-0.644739,0.078290,0.781088,-1.143963,...,2.042412,2.083869,0.025846,-1.570310,1.928149,0.196730,-0.028084,-0.172889,19.193562,7.145782
1965-02-28,0.0,1.0,-0.418118,-0.289938,1.072168,-0.232686,-0.044370,-1.681405,-1.348616,0.593358,...,0.775129,-0.533659,0.442964,0.640847,-0.995475,0.826922,-0.237556,-0.544599,6.191867,2.399321
1965-03-31,0.0,1.0,-0.697797,1.373693,0.570225,-1.181394,0.717623,-0.906386,1.087932,-0.344984,...,1.698642,0.423670,-0.134126,0.859866,0.450620,1.124707,1.290526,0.453862,8.004473,4.470599
1965-04-30,0.0,1.0,-0.002157,0.906231,-0.383899,1.599306,-0.348399,0.206965,0.520725,-1.545488,...,0.126516,-0.622532,0.122027,-0.000870,-0.045935,0.075672,-0.057534,1.840223,22.366516,10.401231
1965-05-31,0.0,1.0,-0.385303,-0.225698,-1.024740,-1.400408,-0.507641,-0.163641,1.676623,1.837518,...,0.312919,0.499158,2.525907,-1.709134,0.251294,0.327740,1.607857,0.144994,25.116323,11.292035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-31,1.0,0.0,0.275840,-0.386375,-0.359091,1.280498,-0.495232,-0.208449,-0.399202,0.826915,...,-0.444555,-0.943215,-0.324993,-0.050399,-0.106444,-0.251992,-1.439031,-0.286482,29.485376,12.738286
2021-09-30,1.0,0.0,-2.708212,-0.802205,-0.620081,-2.854942,-0.815853,-1.863678,-1.050932,-2.148911,...,2.144143,-1.724737,-0.681228,-0.592962,-2.200679,-0.664370,0.742819,-0.981111,35.004150,17.572309
2021-10-31,1.0,0.0,-1.131326,-0.508881,-0.563480,-1.080949,-0.553991,-0.790779,-0.604185,-0.111701,...,0.641133,-1.150639,-0.761134,0.024659,-0.379131,-0.254536,-0.105436,-0.347248,25.289892,10.915476


In [16]:
ggplot(dataset_scaled_x, aes(y='Cl_y')) + geom_violin(quantiles=[.25, .5, .75], quantile_lines=True) + ggtitle("Simplest example")

In [ ]:
# #@title Plot Transformed Data

# def plot_histogram(data: pd.DataFrame):
#     data = data.melt(var_name='param',
#                      value_name='value',
#                      ignore_index=False)

#     return ggplot(data, aes(x='value')) + geom_histogram(aes(y='..density..')) \
#         + geom_density(alpha=.2, color="#de2d26", fill="#ff6666") \
#         + facet_wrap(facets='param', ncol=3, order=1, scales='free', dir='h', )

# plot_histogram(dataset_scaled[columns_to_scale])

In [ ]:
# from lets_plot.bistro.corr import *
# _ = dataset_scaled[columns_to_scale]
# corr_plot(_, threshold=.5).labels(type='lower', diag=True, map_size=True, color='black').tiles(type='lower', diag=True).build()

In [134]:
dataset_scaled_x.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1368 entries, 1965-01-31 to 2021-12-31
Data columns (total 29 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   major_basin_X_East Don River  1368 non-null   float64
 1   major_basin_X_West Don River  1368 non-null   float64
 2   Ca_X                          1368 non-null   float32
 3   Cl_X                          1368 non-null   float32
 4   K_X                           1368 non-null   float32
 5   Mg_X                          1368 non-null   float32
 6   Na_X                          1368 non-null   float32
 7   SO42_X                        1368 non-null   float32
 8   TDS_X                         1368 non-null   float32
 9   ALKT_X                        1368 non-null   float32
 10  CNDT_X                        1368 non-null   float32
 11  HARD_X                        1368 non-null   float32
 12  pH_X                          1368 non-null 

In [ ]:
dataset_scaled_x

In [141]:
from sklearn.model_selection import train_test_split
for target in ['Ca_y', 'Cl_y', 'K_y', 'Mg_y', 'Na_y','SO42_y', 'TDS_y',]:
  # target = 'Ca_y'
  # X = dataset_scaled.copy().drop('rank', axis=1)
  # y = X[target] #X.pop(target)
  # X = X.values.reshape(X.shape[0], X.shape[1], 1)

  # X = dataset_scaled_x[[column for column in dataset_scaled_x.columns if column.endswith("_X")]].copy()
  X = dataset_scaled_x.copy()
  y = X[target]
  n_features = X.shape[1]
  X = X.values.reshape(X.shape[0], 1, X.shape[1])


  sequence_length = 1

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED, shuffle=False)
  # X_train.shape, X_test.shape, y_train.shape, y_test.shape
  loss_fn = 'mse' #tf.keras.losses.Huber(delta=0.5)
  n_months = 25
  model_name = "RNN"
  metric = ['mae', 'mse']

  # sLNN Model----------------------------------------------------------------------------------------------------------
  # def build_model():
  #   """
  #   Builds a single layered neural model.
  #   """
  #   # Create the model
  #   inputs = tf.keras.Input(shape=(X.shape[1:]))
  #   x = inputs
  #   x = tf.keras.layers.Flatten()(x)
  #   # Add the input layer
  #   x = tf.keras.layers.Dense(units=16,activation='elu')(x)
  #   # Add dropout
  #   x = tf.keras.layers.Dropout(rate=0.5)(x)

  #   outputs = tf.keras.layers.Dense(1, activation='linear')(x)

  #   model = tf.keras.Model(inputs, outputs)

  #   # Compile the model
  #   model.compile(loss=loss_fn,
  #                 #optimizer=hp.Choice(name="optimizer", values['rmsprop', 'adam']),
  #                 optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
  #                 metrics=[metric]
  #                 )
  #   return model

  # DNN Model----------------------------------------------------------------------------------------------------------
  # def build_model():
  #   """
  #   Builds a multi-layered neural network model.
  #   """
  #   # Create the model
  #   inputs = tf.keras.Input(shape=(X.shape[1:]))
  #   x = inputs
  #   x = tf.keras.layers.Flatten()(x)
  #   x = tf.keras.layers.Dense(units=64, activation="elu", name=f"dense_L0")(x)
  #   x = tf.keras.layers.Dense(units=64, activation="elu", name=f"dense_L1")(x)
  #   # Tune whether to use dropout before passing it to the output layer.
  #   x = tf.keras.layers.Dropout(rate=0.5)(x)
  #   outputs = tf.keras.layers.Dense(1, activation='linear', name='output')(x)
  #   model = tf.keras.Model(inputs, outputs)

  #   model.compile(loss=loss_fn,
  #                 #optimizer=hp.Choice(name="optimizer", values['rmsprop', 'adam']),
  #                 optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
  #                 metrics=[metric]
  #   )
  #   return model

  # CNN Model----------------------------------------------------------------------------------------------------------
  # def build_model():
  #   """
  #   Builds a convolutional neural network model.
  #   """
  #   inputs = tf.keras.Input(shape=(X.shape[1:]))
  #   x = inputs
  #   # x = tf.keras.layers.Flatten()(x)
  #   x = tf.keras.layers.Conv1D(
  #         filters=32,
  #         kernel_size=12,
  #         activation='elu',
  #         padding="causal", name="conv1d_L0",)(x)
  #   x = tf.keras.layers.MaxPooling1D()(x)
  #   x = tf.keras.layers.Conv1D(
  #         filters=32,
  #         kernel_size=6,
  #         activation='elu',
  #         padding="causal", name="conv1d_L1",)(x)
  #   x = tf.keras.layers.GlobalMaxPooling1D()(x)
  #   # x = tf.keras.layers.Dense(units=16, activation="elu", name=f"dense_L0")(x)
  #   # Tune whether to use dropout before passing it to the output layer.
  #   x = tf.keras.layers.Dropout(0.5)(x)
  #   outputs = tf.keras.layers.Dense(1, activation='linear', name='output')(x)

  #   model = tf.keras.Model(inputs, outputs)

  #   model.compile(loss=loss_fn,
  #                 #optimizer=trial.suggest_categorical("optimizer", ['rmsprop', 'adam']),
  #                 optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
  #                 metrics=[metric]
  #   )
  #   return model

  # AENN Model----------------------------------------------------------------------------------------------------------
  # def build_model():
  #   """
  #   Builds a multi-layered neural network model.
  #   """
  #   # Create the model
  #   inputs = tf.keras.Input(shape=(n_features,))
  #   x = inputs
  #   x = tf.keras.layers.Flatten()(x)
  #   x = tf.keras.layers.Dense(units=64, activation="elu", name=f"dense_L0")(x)
  #   x = tf.keras.layers.Dense(units=32, activation="elu", name=f"dense_L1")(x)
  #   x = tf.keras.layers.Dense(units=16, activation="elu", name=f"dense_L2")(x)
  #   x = tf.keras.layers.Dense(units=32, activation="elu", name=f"dense_L3")(x)
  #   x = tf.keras.layers.Dense(units=64, activation="elu", name=f"dense_L4")(x)
  #   # Tune whether to use dropout before passing it to the output layer.
  #   x = tf.keras.layers.Dropout(rate=0.5)(x)
  #   outputs = tf.keras.layers.Dense(1, activation='linear', name='output')(x)
  #   model = tf.keras.Model(inputs, outputs)

  #   model.compile(loss=loss_fn,
  #                 #optimizer=hp.Choice(name="optimizer", values['rmsprop', 'adam']),
  #                 optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
  #                 metrics=[metric]
  #   )
  #   return model

  # RNN Model----------------------------------------------------------------------------------------------------------
  def build_model():
    """
    Builds a recurrent neural network model.
    """
    inputs = tf.keras.Input(shape=(X.shape[1:]))
    x = inputs
    x = tf.keras.layers.LSTM(
            units=16,
            return_sequences=True,
            # recurrent_dropout=0.2,
            )(x)
    x = tf.keras.layers.LSTM(
            units=16,
            return_sequences=False,
            # recurrent_dropout=0.2,
            )(x)
    x = tf.keras.layers.Dense(units=32, activation='elu', name="dense_L0")(x)
    x = tf.keras.layers.Dense(units=16, activation='elu', name="dense_L1")(x)
    x = tf.keras.layers.Dense(units=32, activation='elu', name="dense_L2")(x)
    x = tf.keras.layers.Dropout(rate=0.5, name="dropout")(x)
    outputs = tf.keras.layers.Dense(1, activation='linear', name='output')(x)

    model = tf.keras.Model(inputs, outputs)

    model.compile(loss=loss_fn,
                  optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                  metrics=metric
    )
    return model

  def get_callbacks(patience, lr_factor):
    '''
    Callbacks used for early stopping and learning rate scheduling.
    '''
    return [
        tf.keras.callbacks.EarlyStopping(
            monitor="val_loss",
            mode="min",
            patience = patience,
            verbose=1),
        # Learning rate is reduced by 'lr_factor' if val_loss stagnates
        # for a number of epochs set with 'patience/2' var.
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss",
            mode="min",
            factor=lr_factor,
            min_lr=1e-6,
            patience=patience//2,
            verbose=1)
        ]

  model = build_model()

  history = model.fit(
      x=X_train,
      y=y_train,
      batch_size=128,
      epochs=2000,
      verbose='auto',
      callbacks=get_callbacks(patience=5, lr_factor=0.3),
      validation_split=0.2,
      shuffle=True,
  )
  val_loss_per_epoch = history.history["val_loss"]
  best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1
  print(f"Best epoch: {best_epoch}")

  # Evaluate the model____________________________________________________________________________________
  eval_result = model.evaluate(X_test, y_test)
  print(f"[Test loss, Test {metric}]: {eval_result}")

  # Plot the model's losses________________________________________________________________________________
  metrics = pd.DataFrame(history.history)
  metrics['epoch'] = history.epoch

  loss = metrics.iloc[best_epoch, metrics.columns.get_loc('loss')]
  val_loss = metrics.iloc[best_epoch, metrics.columns.get_loc('val_loss')]

  data = metrics[['epoch','loss','val_loss']].melt(value_name="value", id_vars="epoch", var_name="loss")

  plot_1 = (
      ggplot(data)
    + geom_line(aes(x="epoch", y='value', color="loss",), size=0.75)
    + theme(legend_position=[1, 1], legend_justification=[1, 1],)
    + geom_vline(xintercept=best_epoch, color="red", linetype="dashed", size=1)
    + labs(title=f"Suitable Model Result for {target} - ({model_name}/{loss_fn})",
          subtitle=f"train loss:{loss:.4f}, val_loss:{val_loss:.4f}, test loss:{eval_result[0]:.4f}",
          x="Epoch", y='Error [%]', color="",
          caption="red dash line: Optimal epoch \n Master Thesis @ Alfred Ofosu, 08/2023")
    + ggsize(800, 400)
    + scale_y_log10()
  )
  ggsave(plot_1, f'losses_{target}_{model_name}_{loss_fn}.png',
        path=f'{local_path}/data/images/CCMEWQI', scale=1.0)


  # Retrain model on the full training data_______________________________________________________________
  hypermodel = build_model()
  hypermodel.fit(
      x=X_train,
      y=y_train,
      batch_size=None,
      epochs=int(best_epoch*1.2),
      verbose='auto',
      callbacks=None,
      validation_data=None,
      shuffle=False,
  )

  hypermodel.save(f'/content/colab_bucket/hyperOpt/CCMEWQI/{target}_{model_name}_{loss_fn}_model.keras')

  # Save best model architecture
  tf.keras.utils.plot_model(hypermodel,
                            f"{local_path}/data/images/CCMEWQI/archiT_{target}_{model_name}_{loss_fn}.jpeg",
                            show_shapes=True)

  # Get the X and y values from the test BatchDataset and predict the last 12 months
  # X = np.concatenate([x for x, y in test_dataset], axis=0)
  # y = np.concatenate([y for x, y in test_dataset], axis=0)
  y_pred = hypermodel.predict(X_test).flatten()

  data = pd.DataFrame({"y":y_test, "y_pred":y_pred},).reset_index()
  data['date'] = pd.to_datetime(data['date'])
  scaler = normalizer(1)
  scaler.fit(dataset_to_model_x[[target]])
  data['y_invs'] = scaler.inverse_transform(data[['y']])
  data['y_pred_invs'] = scaler.inverse_transform(data[['y_pred']])
  plot_2 = (
      ggplot(data.tail(n_months))
    + geom_line(aes(x="date", y="y_invs"),size=1, alpha=1, show_legend=True, color='#1985a1')
    + geom_line(aes(x="date", y="y_pred_invs", ),size=0.8, alpha=0.6, color="red",
                linetype="dashed",
                show_legend=True)
    + labs(title=f"Actual vs. Predicted Values of {target} - ({model_name}/{loss_fn})",
          subtitle=f"Predicited Water Quality over the last {n_months - 1} months",
        x="", y=f'{target} in [%]', color="",
              caption="black line: Actual Values, red dash line: Predicted Values \n Master Thesis @ Alfred Ofosu, 08/2023")
  + scale_x_datetime(format="%b %Y")
  + ggsize(800, 400)
  )
  ggsave(plot_2, f'predicted_{target}_{model_name}_{loss_fn}_{n_months - 1}mons.png',
                    path=f'{local_path}/data/images/CCMEWQI', scale=1.0)
  # Plot model 84 Months_______________________________________________________________
  plot_3 = (
      ggplot(data.tail(85))
    + geom_line(aes(x="date", y="y_invs"),size=1, alpha=1, show_legend=True, color='#1985a1')
    + geom_line(aes(x="date", y="y_pred_invs", ),size=0.8, alpha=0.6, color="red",
                linetype="dashed",
                show_legend=True)
    + labs(title=f"True vs. Predicted Values of {target} - ({model_name}/{loss_fn})",
          subtitle=f"Predicited Water Quality over the last {85 - 1} months",
        x="", y=f'{target} in [%]', color="",
              caption="black line: Actual Values, red dash line: Predicted Values \n Master Thesis @ Alfred Ofosu, 08/2023")
  + scale_x_datetime(format="%b %Y")
  + ggsize(800, 400)
  )

  ggsave(plot_3, f'predicted_{target}_{model_name}_{loss_fn}_84mons.png',
                    path=f'{local_path}/data/images/CCMEWQI', scale=1.0)

  # w, h = 800, 400
  # bunch = GGBunch()
  # bunch.add_plot(plot_1, 0, 0)
  # bunch.add_plot(plot_2, w, 0)
  # bunch.show()

  del hypermodel
  del model

Streaming output truncated to the last 5000 lines.
7/7 [==============================] - 0s 14ms/step - loss: 0.1149 - mae: 0.1983 - mse: 0.1149 - val_loss: 0.0788 - val_mae: 0.2079 - val_mse: 0.0788 - lr: 1.0000e-06
Epoch 278/2000
7/7 [==============================] - 0s 13ms/step - loss: 0.0959 - mae: 0.1979 - mse: 0.0959 - val_loss: 0.0788 - val_mae: 0.2078 - val_mse: 0.0788 - lr: 1.0000e-06
Epoch 279/2000
7/7 [==============================] - 0s 12ms/step - loss: 0.1036 - mae: 0.2019 - mse: 0.1036 - val_loss: 0.0788 - val_mae: 0.2078 - val_mse: 0.0788 - lr: 1.0000e-06
Epoch 280/2000
7/7 [==============================] - 0s 12ms/step - loss: 0.1023 - mae: 0.1920 - mse: 0.1023 - val_loss: 0.0788 - val_mae: 0.2078 - val_mse: 0.0788 - lr: 1.0000e-06
Epoch 281/2000
7/7 [==============================] - 0s 12ms/step - loss: 0.1146 - mae: 0.1940 - mse: 0.1146 - val_loss: 0.0788 - val_mae: 0.2078 - val_mse: 0.0788 - lr: 1.0000e-06
Epoch 282/2000
7/7 [==============================] - 

In [136]:
plot_1

In [137]:
plot_2

In [138]:
plot_3

In [ ]:

data = pd.DataFrame({"y":y_test, "y_pred":y_pred},).reset_index()
data['date'] = pd.to_datetime(data['date'])
scaler = normalizer(1)
scaler.fit(dataset_to_model[[target]])
data['y_invs'] = scaler.inverse_transform(data[['y']])
data['y_pred_invs'] = scaler.inverse_transform(data[['y_pred']])
plot_3 = (
    ggplot(data.tail(85))
  + geom_line(aes(x="date", y="y_invs"),size=1, alpha=1, show_legend=True)
  + geom_line(aes(x="date", y="y_pred_invs", ),size=0.8, alpha=0.6, color="red",
              linetype="dashed",
              show_legend=True)
  + labs(title=f"True vs. Predicted Values of {target} - ({model_name}/{loss_fn})",
        subtitle=f"Predicited Water Quality over the last {85 - 1} months",
      x="", y=f'{target} in [%]', color="",
             caption="black line: Actual Values, red dash line: Predicted Values \n Master Thesis @ Alfred Ofosu, 08/2023")
+ scale_x_datetime(format="%b %Y")
+ ggsize(800, 400)
)

ggsave(plot_3, f'predicted_{target}_{model_name}_{loss_fn}_84mons.png',
                  path=f'{local_path}/data/images/CCMEWQI', scale=1.0)
plot_3

KeyError: ignored

In [ ]:
#@title Prepare Timeseries Data
def create_timeseries_dataset(scaled_dataset:pd.DataFrame,
                              approach_type:str,
                              sequence_length:int,
                              sampling_rate:int, batch_size:int,
                              target_list:list,
                              test_size:int) -> dict:
  dateset_dict = {}
  for target in target_list:
    global n_features
    # Create feature and target values for approach 1
    if approach_type == "all":
      df = scaled_dataset.copy()
      X = df.values.astype(np.float32)
      y = df[target].values.astype(np.float32)
    else:
      # Create feature and target values for approach 2
      X = scaled_dataset[[column for column in scaled_dataset.columns if column.endswith('_X')]]
      X = X.values.astype(np.float32)
      df_target = scaled_dataset[[column for column in scaled_dataset.columns if column.endswith('_y')]]
      y = df_target[target].values.astype(np.float32)

    num_train_samples = int(0.7 * len(X))
    num_val_samples = int(0.2 * len(X))
    # num_test_samples = test_size, #len(X) - num_train_samples - num_val_samples

    sampling_rate = sampling_rate
    sequence_length = sequence_length
    delay = sampling_rate * (sequence_length + 1 - 1)
    batch_size = batch_size

    train_dataset = tf.keras.utils.timeseries_dataset_from_array(
        X[:-delay],
        targets=y[delay:],
        sequence_length=sequence_length,
        batch_size=batch_size,
        shuffle=False,
        seed=SEED,
        start_index=0,
        end_index=num_train_samples,
    )

    val_dataset = tf.keras.utils.timeseries_dataset_from_array(
        X[:-delay],
        targets=y[delay:],
        sequence_length=sequence_length,
        batch_size=batch_size,
        shuffle=False,
        seed=SEED,
        start_index=num_train_samples,
        end_index=num_train_samples+num_val_samples,
    )

    test_dataset = tf.keras.utils.timeseries_dataset_from_array(
        X[:-delay],
        targets=y[delay:],
        sequence_length=sequence_length,
        batch_size=batch_size,
        shuffle=False,
        seed=SEED,
        start_index=num_train_samples+num_val_samples,
    )

    full_dataset = tf.keras.utils.timeseries_dataset_from_array(
        X[:-delay],
        targets=y[delay:],
        sequence_length=sequence_length,
        batch_size=batch_size,
        shuffle=False,
        seed=SEED,
        start_index=0,
        end_index=num_train_samples + num_val_samples,
    )
    # Get the shape for the model
    for samples, targets in train_dataset:
      sequence_length = samples.shape[1]
      n_features = samples.shape[-1]
      break
    dateset_dict[target] = {'full_dataset':full_dataset,
                            'train_dataset': train_dataset,
                            'val_dataset': val_dataset,
                            'test_dataset': test_dataset,
                            'sequence_length':sequence_length,
                            'n_features':n_features}
  return dateset_dict

model_feature_target = create_timeseries_dataset(dataset_scaled,
                                                 approach_type="all",
                                                 sequence_length=1,
                                                 sampling_rate=1,
                                                 batch_size= 128, # * strategy.num_replicas_in_sync,
                                                 target_list=parameters['water quality parameters'],
                                                 test_size=13,
                                                 )

# model_feature_target_x = create_timeseries_dataset(dataset_scaled_x,
#                                                     approach_type="",
#                                                     sequence_length=1,
#                                                     sampling_rate=1,
#                                                     batch_size=256,
#                                                     target_list=parameters['water quality parameters x'],
#                                                     test_size=26,)

In [ ]:
# for target, inner_dict in model_feature_target.items():
#     for key , batch_dataset in inner_dict.items():
#       if key == 'target_dataset' or key == 'val_dataset' or key == 'test_dataset' or key == 'full_dataset':
#         for samples, targets in batch_dataset:
#           print(f"       target:{target}" )
#           print(f"samples shape:{samples.shape}")
#           print(f"targets shape:{targets.shape}\n")
#           break
#         break

In [ ]:
# for target, inner_dict in model_feature_target_x.items():
#   for key , batch_dataset in inner_dict.items():
#     if key == 'target_dataset' or key == 'val_dataset' or key == 'test_dataset':
#       for samples, targets in batch_dataset:
#         print(f"       target:{target}" )
#         print(f"samples shape:{samples.shape}")
#         print(f"targets shape:{targets.shape}\n")
#         break
#       break

       target:Ca_y
samples shape:(256, 1, 15)
targets shape:(256,)

       target:Cl_y
samples shape:(256, 1, 15)
targets shape:(256,)

       target:K_y
samples shape:(256, 1, 15)
targets shape:(256,)

       target:Mg_y
samples shape:(256, 1, 15)
targets shape:(256,)

       target:Na_y
samples shape:(256, 1, 15)
targets shape:(256,)

       target:SO42_y
samples shape:(256, 1, 15)
targets shape:(256,)

       target:TDS_y
samples shape:(256, 1, 15)
targets shape:(256,)



# Tune hyperparameters

Deciding on the number of hyperparameters to tune can be very challenging because the hyperparameter optimizer can not be left to optimize the entire search space as this may cause overfitting. technically, hyperparameter optimization does not optimize the weights of the model so it would make sense to tune hyperparameters that directly/indirectly have an effect on the objective function, the loss function. Other hyperparameters to consider would be the number of layers and neurons. Additionally, the choice of optimization function to use. Bare in mind that the search space grows combinatorially as the size of the hyperparameters grows.

When small training data sizes are fed to large neural networks they tend to overfit and perform poorly on the test data and a well-known technique is to use dropouts[(Hinton et. al, 2012)](https://arxiv.org/pdf/2201.06433.pdf). Dropouts terminate neurons to avoid them from co-adapting features within the neural network layer. The dropout rate
is the fraction of the features that are zeroed out; its usually set between 0.2 and 0.5 (Chollet, 2021).

In [ ]:
approach= "up_down" # @param {type:"string"}
# filepath = f'./data/modelling_data/hyperOpt/keras_tuner/{approach}' # @param {type:"string"}
# gs_filepath = '/content/colab_bucket ' # @param {type:"string"}
gs_filepath = 'gs://master-thesis-yorku-aofosu' # @param {type:"string"}
loss_fn = 'mae' # @param {type:"string"}
model_name = 'rnn' # @param {type:"string"}

if loss_fn == 'mae':
  metric = 'mse'
else: metric = 'mae'

# Tensorflow - Keras Models

In [ ]:
#@title Single-Layered Neural Network (sLNN - Baseline)
# def build_sLnn(hp):
#   """
#   Builds a single layered neural model.
#   """
#   # Create the model
#   inputs = tf.keras.Input(shape=(sequence_length, n_features))
#   x = inputs
#   x = tf.keras.layers.Flatten()(x)
#   # Add the input layer
#   x = tf.keras.layers.Dense(units=hp.Int('num_units', min_value=32, max_value=512, step=32,),
#                             activation=hp.Choice('activation', values=['relu', 'elu', 'selu', 'gelu', 'tanh', ]),
#                             )(x)

#   # Tune whether to use dropout.
#   if hp.Boolean("dropout"):
#       x = tf.keras.layers.Dropout(rate=0.5)(x)

#   outputs = tf.keras.layers.Dense(1, activation='linear')(x)

#   model = tf.keras.Model(inputs, outputs)

#   # Compile the model
#   model.compile(loss=loss_fn,
#                 #optimizer=hp.Choice(name="optimizer", values['rmsprop', 'adam']),
#                 optimizer=tf.keras.optimizers.Adam(),
#                 metrics=[metric]
#                 )
#   return model

# Optuna
def build_sLnn(trial):
  """Builds a single layer neural model."""

  # Create the model
  inputs = tf.keras.Input(shape=(sequence_length, n_features))
  x = inputs
  x = tf.keras.layers.Flatten()(x)
  # Add the input layer
  x = tf.keras.layers.Dense(units=trial.suggest_int(f"units", 32, 128, step=32),
                            activation=trial.suggest_categorical(f"activation", ['elu', 'relu']),
                            name="hidden_layer",
                            )(x)
  # Tune whether to use dropout before passing it to the output layer.
  x = tf.keras.layers.Dropout(rate=trial.suggest_categorical("dropout", [0.2, 0.5]))(x)
  outputs = tf.keras.layers.Dense(1, activation='linear', name='main_output')(x)
  model = tf.keras.Model(inputs, outputs)

  # Compile the model
  model.compile(loss=loss_fn,
                optimizer=tf.keras.optimizers.Adam(learning_rate=trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True)),
                metrics=[metric]
                )
  return model

In [ ]:
#@title Deep Neural Network (DNN)
# # Keras
# def build_dnn(hp):
#   """
#   Builds a multi-layered neural network model.
#   """
#   # Create the model
#   inputs = tf.keras.Input(shape=(sequence_length, n_features))
#   x = inputs
#   x = tf.keras.layers.Flatten()(x)
#   for i in range(hp.Int("dense_layers", min_value=1, max_value=10,)):
#       x = tf.keras.layers.Dense(
#           units=hp.Int("units_L" + str(i), min_value=32, max_value=513, step=32,),
#           activation=hp.Choice('activation' + str(i), values=['relu', 'elu', 'selu', 'gelu', 'tanh']),
#       )(x)
#       # Tune whether to use dropout before passing it to the output layer.
#       if hp.Boolean("dropout_L"+ str(i)):
#           x = tf.keras.layers.Dropout(rate=0.5)(x)
#   outputs = tf.keras.layers.Dense(1, activation='linear')(x)
#   model = tf.keras.Model(inputs, outputs)

#   model.compile(loss=loss_fn,
#                 #optimizer=hp.Choice(name="optimizer", values['rmsprop', 'adam']),
#                 optimizer=tf.keras.optimizers.Adam(),
#                 metrics=[metric]
#   )
#   return model
# Optuna
def build_dnn(trial):
  """
  Builds a multi-layered neural network model.
  """
  # Create the model
  inputs = tf.keras.Input(shape=(sequence_length, n_features))
  x = inputs
  x = tf.keras.layers.Flatten()(x)
  for i in range(trial.suggest_int("n_layers", 1, 5, log=True)):
    x = tf.keras.layers.Dense(
        units=trial.suggest_int(f"units_L{i}", 32, 128, step=32),
        activation=trial.suggest_categorical(f"activation_L{i}", ['elu', 'relu']),
        name=f"dense_{i}")(x)
  # Tune whether to use dropout before passing it to the output layer.
  x = tf.keras.layers.Dropout(rate=trial.suggest_categorical("dropout", [0.2, 0.5]))(x)
  outputs = tf.keras.layers.Dense(1, activation='linear', name='output')(x)
  model = tf.keras.Model(inputs, outputs)

  model.compile(loss=loss_fn,
                #optimizer=hp.Choice(name="optimizer", values['rmsprop', 'adam']),
                optimizer=tf.keras.optimizers.Adam(learning_rate=trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True)),
                metrics=[metric]
  )
  return model

In [ ]:
#@title Convolutional Neural Network (CNN)
# # Keras
# def build_cnn(hp):
#   """
#   Builds a convolutional neural network model.
#   """
#   inputs = tf.keras.Input(shape=(sequence_length, n_features))
#   x = inputs
#   for i in range(hp.Int("conv_layers", min_value=1, max_value=5,)):
#       x = tf.keras.layers.Conv1D(
#           filters=hp.Int("filters_" + str(i), min_value=32, max_value=513, step=32),
#           kernel_size=hp.Int("kernel_size_" + str(i), 3, 5),
#           activation=hp.Choice('activation_conv', values=['relu', 'elu', 'selu', 'gelu', 'tanh', ]),
#           padding="same",
#       )(x)
#   if hp.Choice("global_pooling", ["max", "avg"]) == "max":
#       x = tf.keras.layers.GlobalMaxPooling1D()(x)
#   else:
#       x = tf.keras.layers.GlobalAveragePooling1D()(x)
#     # Tune whether to use dropout before passing it to the output layer.
#   if hp.Boolean("dropout_L"): #+ str(i)
#       x = tf.keras.layers.Dropout(rate=0.5)(x)
#   outputs = tf.keras.layers.Dense(1)(x)

#   model = tf.keras.Model(inputs, outputs)

#   model.compile(loss=loss_fn,
#                 #optimizer=hp.Choice(name="optimizer", values['rmsprop', 'adam']),
#                 optimizer=tf.keras.optimizers.Adam(),
#                 metrics=[metric]
#   )
#   return model

# Optuna
def build_cnn(trial):
  """
  Builds a convolutional neural network model.
  """
  inputs = tf.keras.Input(shape=(sequence_length, n_features))
  x = inputs
  for i in range(trial.suggest_int("n_conv_layers", 1, 5)):
    x = tf.keras.layers.Conv1D(
        filters=trial.suggest_int(f"filters_L{i}", 32, 128, log=True),
        kernel_size=trial.suggest_int(f"kernel_size_L{i}", 2, 3),
        activation=trial.suggest_categorical(f"activation_L{i}", ['elu', 'relu']),
        padding="causal", name=f"conv1d_L{i}",
        # kernel_regularizer=tf.keras.regularizers.L2(0.1),
        # bias_regularizer=tf.keras.regularizers.L2(0.1),
    )(x)
  if trial.suggest_categorical("global_pooling", ["max", "avg"])  == "max":
    x = tf.keras.layers.GlobalMaxPooling1D()(x)
  else:
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
  # Tune whether to use dropout before passing it to the output layer.
  x = tf.keras.layers.Dropout(rate=trial.suggest_categorical("dropout", [0.2, 0.5]))(x)
  outputs = tf.keras.layers.Dense(1, activation='linear', name='output')(x)

  model = tf.keras.Model(inputs, outputs)

  model.compile(loss=loss_fn,
                #optimizer=trial.suggest_categorical("optimizer", ['rmsprop', 'adam']),
                optimizer=tf.keras.optimizers.Adam(learning_rate=trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True)),
                metrics=[metric]
  )
  return model

In [ ]:
#@title Recurrent Neural Network (RNN)
# Keras
# def build_rnn(hp,):
#   """
#   Builds a recurrent neural network model.
#   """
#   inputs = tf.keras.Input(shape=(sequence_length, n_features))
#   x = inputs
#   n_LSTM_layers = hp.Int("n_lstm_layers", min_value=1, max_value=3, default=3)
#   for i in range(n_LSTM_layers):
#     if i == n_LSTM_layers -1:
#       # Add the last LSTM layer without return sequences
#       x = tf.keras.layers.LSTM(
#           units=hp.Int("units_L" + str(i), min_value=32, max_value=513, step=32),
#           return_sequences=False,
#           name=f"lstm_L{i}",
#       )(x)
#     else:
#       # Add the first and intermediate LSTM layers with return sequences
#       x = tf.keras.layers.LSTM(
#           units=hp.Int("units_L" + str(i), min_value=32, max_value=513, step=32),
#           return_sequences=True,
#           name=f"lstm_L{i}",
#           )(x)
#   # Tune whether to use dropout before passing it to the output layer.
#   if hp.Boolean("dropout"):
#       x = tf.keras.layers.Dropout(rate=0.5)(x)
#   outputs = tf.keras.layers.Dense(1)(x)

#   model = tf.keras.Model(inputs, outputs)

#   model.compile(loss=loss_fn,
#                 optimizer=hp.Choice(name="optimizer", values=['rmsprop','adam']),
#                 # optimizer=tf.keras.optimizers.Adam(),
#                 metrics=[metric]
#   )
#   return model

# Optuna
def build_rnn(trial):
  """
  Builds a recurrent neural network model.
  """
  inputs = tf.keras.Input(shape=(sequence_length, n_features))
  x = inputs
  n_LSTM_layers = trial.suggest_int("n_lstm_layers", 1, 2)
  for i in range(n_LSTM_layers):
    if i == n_LSTM_layers -1:
      # Add the last LSTM layer without return sequences
      x = tf.keras.layers.LSTM(
          units=trial.suggest_int(f"units_L{i}",32, 64, log=True),
          return_sequences=False,
          recurrent_dropout=0.5,
          name=f"lstm_L{i}",
      )(x)
    else:
      # Add the first and intermediate LSTM layers with return sequences
      x = tf.keras.layers.LSTM(
          units=trial.suggest_int(f"units_L{i}",32, 64, log=True),
          return_sequences=True,
          recurrent_dropout=0.2,
          name=f"lstm_L{i}",
          )(x)
  # Tune whether to use dropout before passing it to the output layer.
  x = tf.keras.layers.Dropout(rate=trial.suggest_categorical("dropout", [0.2, 0.5]))(x)
  outputs = tf.keras.layers.Dense(1, activation='linear', name='output')(x)

  model = tf.keras.Model(inputs, outputs)

  model.compile(loss=loss_fn,
                optimizer=trial.suggest_categorical("optimizer", ['rmsprop', 'adam']),
                # optimizer=tf.keras.optimizers.Adam(learning_rate=trial.suggest_float("learning_rate", 1e-6, 1e-1, log=True)),
                metrics=[metric]
  )
  return model

In [ ]:
#@title Water Quality Index - Multiclass Prediction
# def build_cnn_rnn(hp,):
#   """
#   Builds a convolutional / recurrent neural network model.
#   """
#   inputs = tf.keras.Input(shape=(sequence_length, n_features))
#   x = inputs
#   # Convolutional Layers
#   for i in range(hp.Int("conv_layers", min_value=1, max_value=5,)):
#       x = tf.keras.layers.Conv1D(
#           filters=hp.Int("filters_" + str(i), min_value=32, max_value=513, step=32),
#           kernel_size=hp.Int("kernel_size_" + str(i), 3, 5),
#           activation=hp.Choice('activation_conv', values=['relu', 'elu', 'selu', 'gelu', 'tanh', ]),
#           padding="same",
#       )(x)
#   if hp.Choice("pooling", ["max", "avg"]) == "max":
#       x = tf.keras.layers.MaxPooling1D()(x)
#   else:
#       x = tf.keras.layers.AveragePooling1D()(x)
#   # Recurrent Neural Network Layers
#   n_LSTM_layers = hp.Int("n_lstm_layers", min_value=1, max_value=5,)
#   for i in range(n_LSTM_layers):
#     if i == n_LSTM_layers - 1:
#       # Add the last LSTM layer without return sequences
#       x = tf.keras.layers.LSTM(
#           units=hp.Int("units_L" + str(i), min_value=32, max_value=513, step=32),
#           return_sequences=False,
#           name=f"lstm_L{i}",
#       )(x)
#     else:
#       # Add the first and intermediate LSTM layers with return sequences
#       x = tf.keras.layers.LSTM(
#           units=hp.Int("units_L" + str(i), min_value=32, max_value=513, step=32),
#           return_sequences=True,
#           name=f"lstm_L{i}",
#           )(x)
#   # Tune whether to use dropout before passing it to the output layer.
#   if hp.Boolean("dropout"): #+ str(i)
#       x = tf.keras.layers.Dropout(rate=0.5)(x)
#   outputs = tf.keras.layers.Dense(1)(x)

#   model = tf.keras.Model(inputs, outputs)

#   model.compile(loss=loss_fn,
#                 #optimizer=hp.Choice(name="optimizer", values['rmsprop', 'adam']),
#                 optimizer=tf.keras.optimizers.Adam(),
#                 metrics=[metric]
#   )
#   return model

In [ ]:
#@title Keras Hyperparameter Optimization
# for target, batch_dataset in model_feature_target.items():
#   train_dataset = batch_dataset['train_dataset']
#   val_dataset = batch_dataset['val_dataset']
#   test_dataset = model_feature_target[target]["test_dataset"]
#   full_dataset = model_feature_target[target]["full_dataset"]
#   sequence_length = batch_dataset['sequence_length']
#   n_features = batch_dataset['n_features']

#   callbacks = [
#       tfdocs.modeling.EpochDots(),
#       tf.keras.callbacks.TensorBoard(
#           log_dir=f"{gs_filepath}/hyperOpt/{approach}/{loss_fn}/{model_name}/logs/{target}_{datetime.now(eastern).strftime('%Y%m%d')}/"
#           ),
#       tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3),
#       ]

#   tuner = kt.Hyperband(
#     hypermodel=desired_model_function,
#     objective=kt.Objective(name="val_loss", direction="min"),
#     max_epochs=50,
#     factor=3,
#     seed=SEED,
#     hyperband_iterations=2,
#     tune_new_entries=True,
#     allow_new_entries=True,
#     directory=f"{gs_filepath}/hyperOpt/{approach}/{loss_fn}/{model_name}/", #f'{gs_filepath}/trials/{loss_fn}/{model_name}/'
#     project_name=f"trail_{target}_{model_name}",
#     overwrite=False,
#     # distribution_strategy=strategy,
#     )

#   # Perform the hyperparameter search
#   tuner.search(
#       train_dataset,
#       validation_data=val_dataset,
#       epochs=50,
#       callbacks=callbacks,
#       verbose=1,
#               )
#   print(f'''
#   ....................tuning of {target} is complete.
#   ''')
#   time.sleep(2)
# #___________________________________________________________________________________________________________________
#   def save_loss_img(history, best_epoch, model_name, loss_fn, target, idx, eval_result):
#     metrics = pd.DataFrame(history.history)
#     metrics['epoch'] = history.epoch

#     loss = metrics.iloc[best_epoch, metrics.columns.get_loc('loss')]
#     val_loss = metrics.iloc[best_epoch, metrics.columns.get_loc('val_loss')]

#     data = metrics[['epoch','loss','val_loss']].melt(value_name="value", id_vars="epoch", var_name="loss")

#     plot = (
#         ggplot(data)
#       + geom_line(aes(x="epoch", y='value', color="loss",), size=0.75)
#       + theme(legend_position=[1, 1], legend_justification=[1, 1],)
#       + geom_vline(xintercept=best_epoch, color="red", linetype="dashed", size=1)
#       + labs(title=f"Best Hypermodel Result ({model_name}/{target}) - {loss_fn.upper()}",
#             subtitle=f"@epoch:{best_epoch}, loss:{loss:.4f}, val_loss:{val_loss:.4f}, test loss*:{eval_result:.4}",
#             x="Epoch", y='Error [mg/L]', color="",
#             caption="* Test loss evaluated on full dataset")
#       + scale_y_log10()
#     )

#     return ggsave(plot, f'{model_name}_losses_{target}.png',
#                   path=f'{local_path}/data/images/{approach}/{loss_fn}/{model_name}', scale=1.0)

#   def get_callbacks(weights_file, patience, lr_factor):
#     '''
#     Callbacks used for saving the best weights, early stopping and learning rate scheduling.
#     '''
#     return [
#         tfdocs.modeling.EpochDots(),
#         # Only save the weights that correspond to the maximum validation accuracy.
#         # tf.keras.callbacks.ModelCheckpoint(
#         #     filepath= weights_file,
#         #     monitor="val_loss",
#         #     mode="min",
#         #     save_best_only=True,
#         #     save_weights_only=True),
#         # If val_loss doesn't improve for a number of epochs set with 'patience' var
#         # training will stop to avoid overfitting.
#         tf.keras.callbacks.EarlyStopping(
#             monitor="val_loss",
#             mode="min",
#             patience = patience,
#             verbose=1),
#         # Learning rate is reduced by 'lr_factor' if val_loss stagnates
#         # for a number of epochs set with 'patience/2' var.
#         tf.keras.callbacks.ReduceLROnPlateau(
#             monitor="val_loss",
#             mode="min",
#             factor=lr_factor,
#             min_lr=1e-6,
#             patience=patience//2,
#             verbose=1)
#         ]

#   # Get the best 4 hyperparameters objects and retrain the model on the training and validation set
#   best_hps = tuner.get_best_hyperparameters(num_trials=3)

#   def get_best_epoch(hp,):

#     model = desired_model_function(hp)
#     history = model.fit(
#         train_dataset,
#         validation_data=val_dataset,
#         epochs=500, # Set the epochs higher than the tunning epochs
#         verbose=0,
#         callbacks=get_callbacks(None,
#                                 patience=5, # Set the patience higher but control is with an early stopping
#                                 lr_factor=0.3),
#         shuffle=False,
#     )
#     val_loss_per_epoch = history.history["val_loss"]
#     best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch))+1
#     print(f"Best epoch: {best_epoch}")
#     return best_epoch, model, history

#   def get_best_trained_model(hp,):
#     best_epoch, model, history = get_best_epoch(hp)
#     model.fit(
#         full_dataset,
#         epochs=int(best_epoch),
#         shuffle=False,
#     )
#     return model, best_epoch, history
#   best_epochs = []
#   best_histories = []
#   best_models = []
#   test_scores = []
#   for idx, hp in enumerate(best_hps):
#     hypermodel, best_epoch, history = get_best_trained_model(hp,)
#     eval_result = hypermodel.evaluate(test_dataset)
#     print(f"{idx}: [Test loss, Test {metric.upper()},]: {eval_result}")
#     best_models.append(hypermodel)
#     test_scores.append(eval_result[0])
#     best_epochs.append(best_epoch)
#     best_histories.append(history)

#     # After training is complete, clear the model and release memory
#     del hypermodel

#   # Print and save jsut the best of the best
#   best_idx = test_scores.index(min(test_scores))
#   print(f"The best model is: {best_idx} with a test score of: {min(test_scores)}")
#   print("\n")
#   best_model = best_models[best_idx]
#   save_loss_img(best_histories[best_idx], best_epochs[best_idx],
#                 model_name, loss_fn, target, best_idx, test_scores[best_idx])
#   tf.keras.utils.plot_model(best_model,
#                             f"{local_path}/data/images/{approach}/{loss_fn}/{model_name}/best_{model_name}_{target}.jpeg",
#                             show_shapes=True)
#   best_model.summary()

#   print(f'''
#   ....................training of {target} is complete.
#   ''')
#   # break
#   # Clear GPU memory if using TensorFlow with GPU
#   tf.keras.backend.clear_session()
#   time.sleep(2)
# # shut down the runtime
# runtime.unassign()

In [ ]:
#@title Hyperparameter Optimization with Optuna

# # Create a dictionary to map model names to the corresponding model building functions
# models = {
#     'sLnn': build_sLnn,
#     'dnn': build_dnn,
#     'cnn': build_cnn,
#     'rnn': build_rnn,
#     # 'cnn_rnn': build_cnn_rnn
# }

# # Select the desired model building function based on the name
# desired_model_function = models[model_name]

# import optuna

# from optuna.integration.tensorboard import TensorBoardCallback
# from optuna.integration import TFKerasPruningCallback
# from optuna.trial import TrialState



# # Objective Function
# def objective(trial, model_name, ds_train, ds_val, sequence_length, n_features):
#   # Clear clutter from previous TensorFlow graphs.
#   tf.keras.backend.clear_session()
#   tf.keras.backend.reset_uids()

#   # Create model instance
#   model = desired_model_function(trial)

#   # Metrics to be monitored by Optuna
#   monitor = "val_loss"

#   # Create callbacks for early stopping and pruning.
#   callbacks = [
#       tf.keras.callbacks.EarlyStopping(patience=3),
#       TFKerasPruningCallback(trial, monitor),
#   ]
#   # Train model.
#   history = model.fit(
#       x=ds_train,
#       y=None,
#       batch_size=None,
#       epochs=100,
#       verbose=0,
#       callbacks=callbacks,
#       validation_data=ds_val,
#       shuffle=False,
#   )

#   return history.history[monitor][-1]
# #_________________________________________________________________________________________________________
# def show_result(study,):
#     import json
#     import pickle

#     pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
#     complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

#     print("\nStudy statistics for {} using {}".format(target, model_name))
#     print("  Number of finished trials: ", len(study.trials))
#     print("  Number of pruned trials: ", len(pruned_trials))
#     print("  Number of complete trials: ", len(complete_trials))

#     print("Best trial:")
#     trial = study.best_trial

#     print("  Value: ", trial.value)

#     print("  Params: ")
#     for key, value in trial.params.items():
#         print("    {}: {}".format(key, value))

#     trial.params["Value"] = trial.value

#     # Save the best parameters as JSON
#     with open(f"/content/colab_bucket/hyperOpt/{approach}/{loss_fn}/{model_name}/optuna_best_hp_{target}_{model_name}.json",'w') as f:
#       json.dump(study.best_trial.params, f, indent=4)

#     # Save the study object
#     with open(f"/content/colab_bucket/hyperOpt/{approach}/{loss_fn}/{model_name}/optuna_best_study_{target}_{model_name}.pkl", 'wb') as f:
#       pickle.dump(study, f)
# #_________________________________________________________________________________________________________
# # An Early Stop Function that terminates the study of trials are being pruned consecutive after n times
# # where n is an int value you set
# class StopWhenTrialKeepBeingPrunedCallback:
#     def __init__(self, threshold: int):
#         self.threshold = threshold
#         self._consequtive_pruned_count = 0

#     def __call__(self, study: optuna.study.Study, trial: optuna.trial.FrozenTrial) -> None:
#         if trial.state == optuna.trial.TrialState.PRUNED:
#             self._consequtive_pruned_count += 1
#         else:
#             self._consequtive_pruned_count = 0

#         if self._consequtive_pruned_count >= self.threshold:
#             study.stop()
# #_________________________________________________________________________________________________________
# def save_loss_img(history, best_epoch, model_name, loss_fn, target, eval_result):
#     metrics = pd.DataFrame(history.history)
#     metrics['epoch'] = history.epoch

#     loss = metrics.iloc[best_epoch, metrics.columns.get_loc('loss')]
#     val_loss = metrics.iloc[best_epoch, metrics.columns.get_loc('val_loss')]

#     data = metrics[['epoch','loss','val_loss']].melt(value_name="value", id_vars="epoch", var_name="loss")

#     plot = (
#         ggplot(data)
#       + geom_line(aes(x="epoch", y='value', color="loss",), size=0.75)
#       + theme(legend_position=[1, 1], legend_justification=[1, 1],)
#       + geom_vline(xintercept=best_epoch, color="red", linetype="dashed", size=1)
#       + labs(title=f"Optimal Hypermodel Result ({model_name}/{target}) - {loss_fn.upper()}",
#             subtitle=f"train loss:{loss:.4f}, val_loss:{val_loss:.4f}, test loss:{eval_result:.4f}",
#             x="Epoch", y='Error [mg/L]', color="",
#             caption="red dash line: Optimal epoch")
#       + scale_y_log10()
#     )

#     return ggsave(plot, f'optuna_{model_name}_losses_{target}.png',
#                   path=f'{local_path}/data/images/{approach}/{loss_fn}/{model_name}', scale=1.0)

# #_________________________________________________________________________________________________________
# def save_true_and_predicted_img(y, ypred, model_name, target,):
#   data = pd.DataFrame({"y":y, "y_pred":y_pred}, index=dataset_to_model_x.tail(len(y)).index).reset_index()
#   data['date'] = pd.to_datetime(data['date'])
#   scaler = normalizer(2)
#   scaler.fit(dataset_to_model_x[[target]])
#   data['y_invs'] = scaler.inverse_transform(data[['y']])
#   data['y_pred_invs'] = scaler.inverse_transform(data[['y_pred']])
#   # data = pd.melt(data, id_vars="date", value_vars=["y","y_pred"])
#   plot = (
#       ggplot(data.tail(13))
#     + geom_line(aes(x="date", y="y_invs"),size=1, alpha=1, show_legend=True)
#     + geom_line(aes(x="date", y="y_pred_invs", ),size=0.8, alpha=0.6, color="red",
#                 linetype="dashed",
#                 show_legend=True)
#     + theme(legend_position=[1, 1], legend_justification=[1, 1],)
#     + labs(title=f"True vs. Predicted Values of {target}. ({model_name}/{loss_fn.upper()})",
#          subtitle=f"Predicited concentrations of {target} over the last 12 months",
#         x="", y=f'{target} in [mg/L]', color="",
#               caption="blue line: True Values\n red dash line: Predicted Values")
#   + scale_x_datetime(format="%b %Y")
#   )
#   return ggsave(plot, f'optuna_{model_name}_predicted_{target}.png',
#                   path=f'{local_path}/data/images/{approach}/{loss_fn}/{model_name}', scale=1.0)
# #_________________________________________________________________________________________________________
# for target, batch_dataset in model_feature_target_x.items():
#   # if target == "Na_y" or target == "SO42_y" or target == "TDS_y":
#   train_dataset = batch_dataset['train_dataset']
#   val_dataset = batch_dataset['val_dataset']
#   test_dataset = batch_dataset["test_dataset"]
#   full_dataset = batch_dataset["full_dataset"]
#   sequence_length = batch_dataset['sequence_length']
#   n_features = batch_dataset['n_features']

#   study = optuna.create_study(
#       direction="minimize",
#       sampler=optuna.samplers.TPESampler(seed=SEED),
#       pruner=optuna.pruners.HyperbandPruner(min_resource=1, max_resource='auto'),
#       study_name="optuna_{}_using_{}".format(target, model_name),
#       load_if_exists=False,
#   )
#   study_stop_cb = StopWhenTrialKeepBeingPrunedCallback(10)
#   study.optimize(lambda trial: objective(trial, model_name, train_dataset, val_dataset, sequence_length, n_features),
#                 n_trials=100,
#                 #  n_jobs=-1,
#                 timeout=60*60,
#                 gc_after_trial=False, # This avoids running out of memory if set to True (a performance trade-off!)
#                 show_progress_bar=True,
#                 callbacks=[study_stop_cb]
#                 )
#   # Show study results
#   show_result(study)

#   def get_callbacks(patience, lr_factor):
#     '''
#     Callbacks used for saving the best weights, early stopping and learning rate scheduling.
#     '''
#     return [
#         # tfdocs.modeling.EpochDots(),
#         tf.keras.callbacks.EarlyStopping(
#             monitor="val_loss",
#             mode="min",
#             patience = patience,
#             verbose=1),
#         # Learning rate is reduced by 'lr_factor' if val_loss stagnates
#         # for a number of epochs set with 'patience/2' var.
#         tf.keras.callbacks.ReduceLROnPlateau(
#             monitor="val_loss",
#             mode="min",
#             factor=lr_factor,
#             min_lr=1e-6,
#             patience=patience//2,
#             verbose=1)
#         ]
#   # Get the best epoch
#   best_model = desired_model_function(study.best_trial)
#   history = best_model.fit(
#       x=train_dataset,
#       y=None,
#       batch_size=None,
#       epochs=1000,
#       verbose='auto',
#       callbacks=get_callbacks(patience=5, lr_factor=0.3),
#       validation_data=val_dataset,
#       shuffle=False,
#   )
#   val_loss_per_epoch = history.history["val_loss"]
#   best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch))+1
#   print(f"Best epoch: {best_epoch}")

#   # Save best model architecture
#   tf.keras.utils.plot_model(best_model,
#                             f"{local_path}/data/images/{approach}/{loss_fn}/{model_name}/best_optuna_{model_name}_{target}.jpeg",
#                             show_shapes=True)
#   # Show summary
#   best_model.summary()

#   # After training is complete, clear the model and release memory
#   del best_model

#   # retrain the model on the full training dataset
#   hypermodel = desired_model_function(study.best_trial)
#   hypermodel.fit(
#       x=full_dataset,
#       y=None,
#       batch_size=None,
#       epochs=int(best_epoch*1.2),
#       verbose='auto',
#       callbacks=None,
#       validation_data=None,
#       shuffle=False,
#   )

#   eval_result = hypermodel.evaluate(test_dataset)
#   print(f"[Test loss, Test {metric.upper()}]: {eval_result}")
#   # Save the loss and validation plot
#   save_loss_img(history, best_epoch, model_name, loss_fn, target, eval_result[0])

#   # Get the X and y values from the test BatchDataset and predict the last 12 months
#   X = np.concatenate([x for x, y in test_dataset], axis=0)
#   y = np.concatenate([y for x, y in test_dataset], axis=0)
#   y_pred = hypermodel.predict(X).flatten()

#   # Save the drafted predictions
#   save_true_and_predicted_img(y, y_pred, model_name, target,)

#   # After training is complete, clear the model and release memory
#   del hypermodel
# # shut down the runtime
# # runtime.unassign()

In [ ]:
#@title Manual Hyperparamter Optimization

# def build_rnn_model():
#   """
#   Builds a recurrent neural network model.
#   """
#   inputs = tf.keras.Input(shape=(sequence_length, n_features))
#   x = inputs
#   x = tf.keras.layers.LSTM(
#           units=128,
#           return_sequences=True,
#           # dropout=0.2,
#           # recurrent_dropout=0.2,
#           name=f"lstm_L1",
#           )(x)

#   # x = tf.keras.layers.LSTM(
#   #         units=128,
#   #         return_sequences=True,
#   #         recurrent_dropout=0.2,
#   #         name=f"lstm_L2",
#   #         )(x)

#   x = tf.keras.layers.LSTM(
#           units=64,
#           return_sequences=False,
#           dropout=0.2,
#           recurrent_dropout=0.2,
#           name=f"lstm_L2",
#       )(x)
#   x = tf.keras.layers.Dropout(rate=0.5, name="dropout")(x)
#   x = tf.keras.layers.Dense(units=64, name="dense_L1")(x)

#   outputs = tf.keras.layers.Dense(1)(x)

#   model = tf.keras.Model(inputs, outputs)

#   model.compile(loss=loss_fn,
#                 #optimizer=trial.suggest_categorical("optimizer", ['rmsprop', 'adam']),
#                 optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
#                 metrics=[metric]
#   )
#   return model

# mmodel = build_rnn_model()

# target ="Cl_y"

# train_dataset = model_feature_target_x[target]['train_dataset']
# val_dataset = model_feature_target_x[target]['val_dataset']
# test_dataset = model_feature_target_x[target]["test_dataset"]
# full_dataset = model_feature_target_x[target]["full_dataset"]
# sequence_length = model_feature_target_x[target]['sequence_length']
# n_features = model_feature_target_x[target]['n_features']

# def get_callbacks(patience, lr_factor):
#   '''
#   Callbacks used for saving the best weights, early stopping and learning rate scheduling.
#   '''
#   return [
#       # tfdocs.modeling.EpochDots(),
#       tf.keras.callbacks.EarlyStopping(
#           monitor="val_loss",
#           mode="min",
#           patience = patience,
#           verbose=1),
#       # Learning rate is reduced by 'lr_factor' if val_loss stagnates
#       # for a number of epochs set with 'patience/2' var.
#       tf.keras.callbacks.ReduceLROnPlateau(
#           monitor="val_loss",
#           mode="min",
#           factor=lr_factor,
#           min_lr=1e-5,
#           patience=patience//2,
#           verbose=1)
#       ]

# history = mmodel.fit(
#     x=train_dataset,
#     y=None,
#     batch_size=None,
#     epochs=1000,
#     verbose='auto',
#     callbacks=get_callbacks(patience=5, lr_factor=0.3),
#     validation_data=val_dataset,
#     shuffle=False,
# )
# val_loss_per_epoch = history.history["val_loss"]
# best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch))+1
# print(f"Best epoch: {best_epoch}")



# eval_result = mmodel.evaluate(test_dataset)
# print(f"[Test loss, Test {metric.upper()}]: {eval_result}")

# metrics = pd.DataFrame(history.history)
# metrics['epoch'] = history.epoch

# loss = metrics.iloc[best_epoch, metrics.columns.get_loc('loss')]
# val_loss = metrics.iloc[best_epoch, metrics.columns.get_loc('val_loss')]

# data = metrics[['epoch','loss','val_loss']].melt(value_name="value", id_vars="epoch", var_name="loss")

# plot_1 = (
#     ggplot(data)
#   + geom_line(aes(x="epoch", y='value', color="loss",), size=0.75)
#   + theme(legend_position=[1, 1], legend_justification=[1, 1],)
#   + geom_vline(xintercept=best_epoch, color="red", linetype="dashed", size=1)
#   + labs(title=f"Optimal Hypermodel Result ({model_name}/{target}) - {loss_fn.upper()}",
#         subtitle=f"train loss:{loss:.4f}, val_loss:{val_loss:.4f}, test loss:{eval_result[0]:.4f}",
#         x="Epoch", y='Error [mg/L]', color="",
#         caption="red dash line: Optimal epoch")
#   + scale_y_log10()
# )

# plot_1

# hypermodel = build_rnn_model()
# hypermodel.fit(
#     x=full_dataset,
#     y=None,
#     batch_size=None,
#     epochs=int(best_epoch*1.2),
#     verbose='auto',
#     callbacks=None,
#     validation_data=None,
#     shuffle=False,
# )

# # Get the X and y values from the test BatchDataset and predict the last 12 months
# X = np.concatenate([x for x, y in test_dataset], axis=0)
# y = np.concatenate([y for x, y in test_dataset], axis=0)
# y_pred = hypermodel.predict(X).flatten()

# data = pd.DataFrame({"y":y, "y_pred":y_pred}, index=dataset_to_model_x.tail(len(y)).index).reset_index()
# data['date'] = pd.to_datetime(data['date'])
# scaler = normalizer(2)
# scaler.fit(dataset_to_model_x[['Ca_X']])
# data['y_invs'] = scaler.inverse_transform(data[['y']])
# data['y_pred_invs'] = scaler.inverse_transform(data[['y_pred']])
# plot_2 = (
#     ggplot(data.tail(13))
#   + geom_line(aes(x="date", y="y_invs"),size=1, alpha=1, show_legend=True)
#   + geom_line(aes(x="date", y="y_pred_invs", ),size=0.8, alpha=0.6, color="red",
#               linetype="dashed",
#               show_legend=True)
#   + theme(legend_position=[1, 1], legend_justification=[1, 1],)
#   + labs(title=f"True vs. Predicted Values of {target}. ({model_name}/{loss_fn.upper()})",
#         subtitle=f"Predicited concentrations of {target} over the last 12 months",
#       x="", y=f'{target} in [mg/L]', color="",
#             caption="blue line: True Values\n red dash line: Predicted Values")
# + scale_x_datetime(format="%b %Y")
# )
# plot_2

In [ ]:
# Load the TensorBoard notebook extension.
# %load_ext tensorboard

# %tensorboard --logdir "/content/drive/MyDrive/hyperOpt-trials/all/tensorboard/tb_logs/mae/cnn/20230703-051156"

In [ ]:
#https://www.youtube.com/watch?v=KrbV75Mby5E&t=149s

In [ ]:
#@title Hyperparameter Optimization with Optuna - Old

# import tensorflow_docs as tfdocs
# import tensorflow_docs.modeling

# import warnings
# from tensorflow import get_logger
# get_logger().setLevel('ERROR')
# warnings.filterwarnings("ignore", message="Setting the random state for TF")

# import absl.logging
# absl.logging.set_verbosity(absl.logging.ERROR)

# import optuna

# from optuna.integration.tensorboard import TensorBoardCallback
# from optuna.integration import TFKerasPruningCallback
# from optuna.trial import TrialState

# from sklearn.model_selection import TimeSeriesSplit
# from sklearn.model_selection import cross_validate
# from scikeras.wrappers import KerasRegressor
# import IPython
# import json
# import time

# global filepath, loss_fn, model_name, target

# # Optimizer
# # def create_optimizer(trial):
# #     # We optimize the choice of optimizers as well as their parameters.
# #     kwargs = {}
# #     optimizer_selected = trial.suggest_categorical("optimizer", [ "Adam", "RMSprop", "SGD", ])
# #     if optimizer_selected == "Adam":
# #         kwargs["learning_rate"] = trial.suggest_float("adam_learning_rate", 1e-4, 1e-1, log=True)
# #     elif optimizer_selected == "RMSprop":
# #         kwargs["learning_rate"] = trial.suggest_float(
# #             "rmsprop_learning_rate", 1e-4, 1e-1, log=True
# #         )
# #         kwargs["momentum"] = trial.suggest_float("rmsprop_momentum", 1e-5, 1e-1, log=True)
# #     elif optimizer_selected == "SGD":
# #         kwargs["learning_rate"] = trial.suggest_float(
# #             "sgd_opt_learning_rate", 1e-2, 1e-1, log=True
# #         )
# #         kwargs["momentum"] = trial.suggest_float("sgd_opt_momentum", 1e-5, 1e-1, log=True)

# #     optimizer = getattr(tf.keras.optimizers, optimizer_selected)(**kwargs)
# #     return optimizer
# # END_____________________________________________________________________________________________________
# # A single layer neural network
# def build_sLnn(trial):
#   """Builds a single layer neural model."""

#   # Create the model
#   inputs = tf.keras.Input(shape=(sequence_length, n_features))
#   x = inputs
#   x = tf.keras.layers.Flatten()(x)
#   # Add the input layer
#   x = tf.keras.layers.Dense(trial.suggest_int('num_units', 32, 512, step=32,),
#                             activation=trial.suggest_categorical('activation', values=['relu', 'elu', 'selu', 'gelu', 'tanh', ]),
#                             )(x)

#   # Tune whether to use dropout.
#   if hp.Boolean("dropout"):
#       x = tf.keras.layers.Dropout(rate=0.5)(x)

#   outputs = tf.keras.layers.Dense(1, activation='linear')(x)

#   model = tf.keras.Model(inputs, outputs)

#   # Compile the model
#   model.compile(loss=loss_fn,
#                 optimizer=tf.keras.optimizers.Adam(learning_rate=hp.Float("learning_rate", 1e-6, 1e-1, sampling="log")),
#                 metrics=['mae', 'mse']
#                 )
#   return model
# # END_____________________________________________________________________________________________________
# # Deep Neural Network (DNN)

# def dnn_builder(trial,):
#     dnn = tf.keras.Sequential()
#     dnn.add(tf.keras.Input(shape=(X.shape[1:])))
#     for i in range(trial.suggest_int('n_dense_layers', 1, 10)):
#         units = trial.suggest_int(f'n_units_dense_L{i}', 32, 512, step=32)
#         activation = trial.suggest_categorical(f'activation_dense_L{i}', ['relu', 'elu',])
#         weight_decay = trial.suggest_float(f'weight_decay_dense_L{i}', 1e-5, 1e-1, log=True)
#         dropout_rate = trial.suggest_float(f'dropout_dense_L{i}', 0.1, 0.5, log=True)

#         dnn.add(tf.keras.layers.Dense(
#             units=units,
#             activation=activation,
#             kernel_regularizer=tf.keras.regularizers.l2(weight_decay),
#             name=f'dense_L{i}',
#         ))
#         dnn.add(tf.keras.layers.Dropout(dropout_rate, name=f'dropout_dense_L{i}'))

#     dnn.add(tf.keras.layers.Dense(1, name="output_L"))
#     dnn.compile(optimizer=tf.keras.optimizers.Adam(
#         learning_rate=trial.suggest_float("adam_learning_rate", 1e-4, 1e-1, log=True)),
#             loss=loss_fn,
#             metrics=['mae', 'mse', ]
#         )
#     return dnn

# # END_____________________________________________________________________________________________________

# # Covolutional Neural Network (CNN)
# # noinspection PyShadowingNames
# def cnn_builder(trial,):

#     cnn = tf.keras.Sequential()
#     cnn.add(tf.keras.Input(shape=(X.shape[1:])))
#     for j in range(trial.suggest_int("n_conv1d_layers", 1, 5)):
#         n_filters = trial.suggest_int(f"filters_conv1d_L{j}", 32, 512, step=32)
#         kernel_size = trial.suggest_int(f"kernel_size_conv1d_L{j}", 1, 10)
#         weight_decay = trial.suggest_float(f"weight_decay_conv1d_L{j}", 1e-5, 1e-1, log=True)
#         activation = trial.suggest_categorical(f"activation_conv1d_L{j}", ['relu', 'elu',])
#         pool_size = trial.suggest_int(f"pool_size_conv1d_L{j}", 1, 10)

#         cnn.add(tf.keras.layers.Conv1D(
#             filters=n_filters,
#             padding='same',
#             kernel_size=kernel_size,
#             activation=activation,
#             kernel_regularizer=tf.keras.regularizers.l2(weight_decay),
#             name="conv1d_L{}".format(j),
#             )
#         )
#         cnn.add(tf.keras.layers.MaxPooling1D(pool_size=pool_size, padding='same', name=f"max_pooling1d_{j}", ))
#     # Flatten the Convolutional Output (Matrix) to a one dimensional vector
#     cnn.add(tf.keras.layers.Flatten())

#     # Fully connected layers
#     for i in range(trial.suggest_int('n_dense_layers', 1, 5)):
#         units = trial.suggest_int(f'n_units_dense_L{i}', 32, 512, step=32)
#         activation = trial.suggest_categorical(f'activation_dense_L{i}', ['relu', 'elu',])
#         weight_decay = trial.suggest_float(f'weight_decay_dense_L{i}', 1e-5, 1e-1, log=True)
#         dropout_rate = trial.suggest_float(f'dropout_dense_L{i}', 0.1, 0.5, log=True)

#         cnn.add(tf.keras.layers.Dense(
#             units=units,
#             activation=activation,
#             kernel_regularizer=tf.keras.regularizers.l2(weight_decay),
#             name=f'dense_L{i}',
#         ))
#         cnn.add(tf.keras.layers.Dropout(dropout_rate, name=f'dropout_dense_L{i}'))
#     cnn.add(tf.keras.layers.Dense(1, name="output_L"))
#     cnn.compile(optimizer=tf.keras.optimizers.Adam(
#         learning_rate=trial.suggest_float("adam_learning_rate", 1e-4, 1e-1, log=True)),
#             loss=loss_fn,
#             metrics=['mae', 'mse', ]
#         )
#     return cnn
# # END_____________________________________________________________________________________________________

# # Recurrent Neural Network (RNN)
# # noinspection PyShadowingNames

# def rnn_builder(trial,):
#     # We optimize the numbers of layers and their units.
#     n_layers = trial.suggest_int("n_lstm_layers", 1, 5)
#     rnn = tf.keras.Sequential()
#     rnn.add(tf.keras.Input(shape=(X.shape[1:])))
#     for k in range(n_layers):

#         # Define the hyperparameters to be tuned
#         units =trial.suggest_int(f"lstm_units_L{k}", 32, 512, log=True)

#         if k == n_layers - 1:
#             # Add the last LSTM layer without return sequences
#             rnn.add(tf.keras.layers.LSTM(units=units,
#                                          return_sequences=False,
#                                          name=f"lstm_L{k}",
#                                          recurrent_dropout=0.2,
#                                             )
#             )
#         else:
#             # Add the first and intermediate LSTM layers with return sequences
#             rnn.add(tf.keras.layers.LSTM(units=units,
#                                          recurrent_dropout=0.2,
#                                          return_sequences=True,
#                                          name=f"lstm_L{k}",
#                                             )
#             )
#     rnn.add(tf.keras.layers.Dropout(trial.suggest_float(f'dropout_dense_Lx', 0.1, 0.5, log=True)))
#     # Fully connected layers
#     for i in range(trial.suggest_int('n_dense_layers', 1, 5)):
#         units = trial.suggest_int(f'n_units_dense_L{i}', 32, 512, log=True)
#         activation = trial.suggest_categorical(f'activation_dense_L{i}', ['relu', 'elu',])
#         weight_decay = trial.suggest_float(f'weight_decay_dense_L{i}', 1e-5, 1e-1, log=True)
#         dropout_rate = trial.suggest_float(f'dropout_dense_L{i}', 0.1, 0.5, log=True)

#         rnn.add(tf.keras.layers.Dense(
#             units=units,
#             activation=activation,
#             kernel_regularizer=tf.keras.regularizers.l2(weight_decay),
#             name=f'dense_L{i}',
#         ))
#         rnn.add(tf.keras.layers.Dropout(dropout_rate, name=f'dropout_dense_L{i}'))
#     rnn.add(tf.keras.layers.Dense(1, name="output_L"))
#     rnn.compile(optimizer=tf.keras.optimizers.Adam(
#         learning_rate=trial.suggest_float("adam_learning_rate", 1e-4, 1e-1, log=True)),
#             loss=loss_fn,
#             metrics=['mae', 'mse', ]
#         )

#     return rnn
# # END_____________________________________________________________________________________________________

# # Convolutional Neural Network + Recurrent Neural Network (CNN+RNN)
# # noinspection PyShadowingNames
# def cnn_rnn_builder(trial,):

#     cnn_rnn = tf.keras.Sequential()
#     cnn_rnn.add(tf.keras.Input(shape=(X.shape[1:])))
#     for l in range(trial.suggest_int("n_cnn_layers", 1, 5)):
#         n_filters = trial.suggest_int(f"filters_conv1d_L{l}", 32, 512, step=32)
#         kernel_size = trial.suggest_int(f"kernel_size_conv1d_L{l}", 1, 10)
#         weight_decay = trial.suggest_float(f"weight_decay_conv1d_L{l}", 1e-5, 1e-1, log=True)
#         activation = trial.suggest_categorical(f"activation_conv1d_L{l}", ['relu', 'elu',])
#         pool_size = trial.suggest_int(f"pool_size_conv1d_L{l}", 1, 10)


#         cnn_rnn.add(tf.keras.layers.Conv1D(
#             filters=n_filters,
#             padding='same',
#             kernel_size=kernel_size,
#             activation=activation,
#             kernel_regularizer=tf.keras.regularizers.l2(weight_decay),
#             name="con1d_L{}".format(l),
#         )
#         )
#         cnn_rnn.add(tf.keras.layers.MaxPooling1D(pool_size=pool_size,
#                                                  padding='same',
#                                                  name=f"max_pooling1d_L{l}", ))
#     cnn_rnn.add(tf.keras.layers.Flatten())
#     n_rnn_layers = trial.suggest_int("n_rnn_layers", 1, 5)
#     for m in range(n_rnn_layers):

#         # Define the hyperparameters to be tuned
#         units =trial.suggest_int(f"lstm_units_L{m}", 32, 512, step=32)
#         weight_decay = trial.suggest_float(f"weight_decay_lstm_L{m}", 1e-5, 1e-1, log=True)
#         dropout_rate = trial.suggest_float(f"lstm_dropout_lstm_L{m}", 0.1, 0.5, log=True)

#         if m == n_rnn_layers - 1:
#             # Add the last LSTM layer without return sequences
#             cnn_rnn.add(tf.keras.layers.LSTM(units=units,
#                                         kernel_regularizer=tf.keras.regularizers.l2(weight_decay),
#                                         dropout=dropout_rate,
#                                         return_sequences=False,
#                                         name=f"lstm_L{m}",
#                                         )
#             )
#         else:
#             # Add the first and intermediate LSTM layers with return sequences
#             cnn_rnn.add(tf.keras.layers.LSTM(units=units,
#                                         kernel_regularizer=tf.keras.regularizers.l2(weight_decay),
#                                         dropout=dropout_rate,
#                                         return_sequences=True,
#                                         name=f"lstm_L{m}",
#                                         )
#             )
#     cnn_rnn.add(tf.keras.layers.Flatten())
#     # Fully connected layers
#     for i in range(trial.suggest_int('n_dense_layers', 1, 5)):
#         units = trial.suggest_int(f'n_units_dense_L{i}', 32, 512, step=32)
#         activation = trial.suggest_categorical(f'activation_dense_L{i}', ['relu', 'elu',])
#         weight_decay = trial.suggest_float(f'weight_decay_dense_L{i}', 1e-5, 1e-1, log=True)
#         dropout_rate = trial.suggest_float(f'dropout_dense_L{i}', 0.1, 0.5, log=True)

#         cnn_rnn.add(tf.keras.layers.Dense(
#             units=units,
#             activation=activation,
#             kernel_regularizer=tf.keras.regularizers.l2(weight_decay),
#             name=f'dense_L{i}',
#         ))
#         cnn_rnn.add(tf.keras.layers.Dropout(dropout_rate, name=f'dropout_dense_L{i}'))
#     cnn_rnn.add(tf.keras.layers.Dense(1, name="output_L"))
#     cnn_rnn.compile(optimizer=tf.keras.optimizers.Adam(
#         learning_rate=trial.suggest_float("adam_learning_rate", 1e-4, 1e-1, log=True)),
#             loss=loss_fn,
#             metrics=['mae', 'mse', ]
#         )

#     return cnn_rnn
# # END_____________________________________________________________________________________________________

# # noinspection PyShadowingNames
# def show_result(study,):
#     import json
#     import pickle

#     pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
#     complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

#     print("\nStudy statistics for {} using {}".format(target, model_name))
#     print("  Number of finished trials: ", len(study.trials))
#     print("  Number of pruned trials: ", len(pruned_trials))
#     print("  Number of complete trials: ", len(complete_trials))

#     print("Best trial:")
#     trial = study.best_trial

#     print("  Value: ", trial.value)

#     print("  Params: ")
#     for key, value in trial.params.items():
#         print("    {}: {}".format(key, value))

#     trial.params["Value"] = trial.value

#     # Save the best parameters as JSON
#     with open(f'{filepath}/best_hps/{loss_fn}/{model_name}/best_hp_{target}_{model_name}.json','w') as f:
#       json.dump(study.best_trial.params, f, indent=4)

#     # Save the study object
#     with open(f'{filepath}/studies/{loss_fn}/{model_name}/best_study_{target}_{model_name}.pkl', 'wb') as f:
#       pickle.dump(study, f)
# #_________________________________________________________________________________________________________
# # Objective function can take additional arguements
# # https://optuna.readthedocs.io/en/stable/faq.html#objective-func-additional-args

# # noinspection PyRedeclaration
# def objective(trial, model_name, X, y):

#      # Clear clutter from previous TensorFlow graphs.
#     tf.keras.backend.clear_session()

#     global model

#     from sklearn.model_selection import TimeSeriesSplit
#     from scikeras.wrappers import KerasRegressor

#     tscv = TimeSeriesSplit(
#             n_splits=5,
#             gap=0,
#             max_train_size=None,
#             test_size=None,
#         )

#     # Metrics to be monitored by Optuna.
#     monitor = 'loss'

#     if model_name == "dnn": model = dnn_builder(trial)
#     elif model_name == "cnn": model = cnn_builder(trial)
#     elif model_name == "rnn": model = rnn_builder(trial)
#     elif model_name == "cnn_rnn": model = cnn_rnn_builder(trial)


#     estimator =  KerasRegressor(
#         model=model,
#         random_state=SEED,
#         batch_size=128,
#         verbose=0,
#         callbacks=[
#             # tfdocs.modeling.EpochDots(),
#             tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3),
#             TFKerasPruningCallback(trial, monitor)],
#         shuffle=False,
#         run_eagerly=False,
#         epochs=100, #trial.suggest_int(f'epochs', 1, 1000, step=100),
#         # meta = {"input_shape": input_shape, "sequence_length": sequence_length},
#     )

#     cv_results = cross_validate(
#         estimator,
#         X=X,
#         y=y,
#         cv=tscv,
#         scoring=["neg_mean_absolute_error", "neg_mean_squared_error","neg_root_mean_squared_error"],
#         error_score='raise',
#     )

#     mae = -cv_results["test_neg_mean_absolute_error"]
#     mse = -cv_results["test_neg_mean_squared_error"]
#     rmse = -cv_results["test_neg_root_mean_squared_error"]

#     print(
#         f"Mean Absolute Error:     {mae.mean():.3f} +/- {mae.std():.3f}\n"
#         f"Mean Squared Error:      {mse.mean():.3f} +/- {mse.std():.3f}\n"
#         f"Root Mean Squared Error: {rmse.mean():.3f} +/- {rmse.std():.3f}"
#     )

#     if loss_fn == 'mae':
#         return np.mean(mae)
#     elif loss_fn == 'mae':
#         return np.mean(mse)
# #_________________________________________________________________________________________________________
# # noinspection PyShadowingNames
# for loss_fn in ['mae', 'mse']:
#   if loss_fn == 'mae':
#     for model_name, model_data in model_feature_target.items():
#       if model_name == 'rnn':
#         for target, data in model_data.items():
#           if target == "SO4" or target == "TDS":
#             X, y = data['X'], data['y']
#             filepath = '/content/drive/MyDrive/optuna-studies/all'
#             study = optuna.create_study(
#                 direction="minimize",
#                 sampler=optuna.samplers.TPESampler(seed=SEED),
#                 pruner=optuna.pruners.HyperbandPruner(min_resource=1, max_resource='auto'),
#                 study_name="study_to_predict_{}_using_{}".format(target, model_name),
#             )
#             study.optimize(lambda trial: objective(trial, model_name, X, y),
#                           n_trials=15,
#                           n_jobs=-1,
#                           timeout=60*60,
#                             gc_after_trial=True, # This avoids running out of memory if set to True (a performance trade-off!)
#                             show_progress_bar=True)

#             show_result(study)
# import time
# time.sleep(10)
# for loss_fn in ['mae', 'mse']:
#   for model, model_data in model_feature_target_x.items():
#       for param, data in model_data.items():
#           target = param
#           X, y = data['X'], data['y']
#           filepath = '/content/drive/MyDrive/optuna-studies/up_down'
#           study = optuna.create_study(
#               direction="minimize",
#               sampler=optuna.samplers.TPESampler(seed=42),
#               pruner=optuna.pruners.HyperbandPruner(min_resource=1, max_resource='auto'),
#               study_name="study_to_predict_{}_using_{}".format(target, model_name),
#           )
#           study.optimize(lambda trial: objective(trial, loss_fn, model_name, target, X, y),
#                         n_trials=25,
#                         n_jobs=-1,
#                         timeout=60*60)

#           show_result(study,loss_fn, model_name, target,)